# 12.FP growth算法

## 12.1.FP树节点的建立

In [1]:
class treeNode:
    """
    FP树节点的类
    继承：
        无
    方法：
        __init__ -- 初始化类变量
        inc -- 给count增加关键词出现的次数
        disp -- 以文本方式显示树
    """
    def __init__(self, nameValue, numOccur, parentNode):
        """
        初始化类变量
        参数：
            self -- 自身，treeNode类实体
            nameValue -- 名字，字符串
            numOccur -- 出现的频率，整数
            parentNode -- 父节点，treeNode类实体
        返回：
            无
        """
        #各种初始化赋值
        self.name = nameValue
        self.count = numOccur
        #连接关系初始化为无
        self.nodeLink = None
        #记录父节点
        self.parent = parentNode
        #初始化子节点为空字典
        self.children = {} 
    
    def inc(self, numOccur):
        """
        增加关键词出现的次数
        参数：
            self -- 自身，treeNode类实体
            numOccur -- 关键词出现的次数
        返回：
            无
        """
        self.count += numOccur
        
    def disp(self, ind=1):
        """
        以文本的形式显示树
        参数：
            self -- 自身，treeNode类实体
            ind -- 层索引，非负整数，默认为1
        返回：
            无
        """
        print('  '*ind, self.name, ' ', self.count)
        #递归调用dips，显示子树
        for child in self.children.values():
            child.disp(ind+1)

测试

In [2]:
#建立根节点
rootNode = treeNode('pyramid', 9, None)
#建立一个子节点
rootNode.children['eye'] = treeNode('eye', 13, None)
#显示结果
rootNode.disp()
#再建立一个子节点
rootNode.children['phoenix'] = treeNode('phoenix', 3, None)
#再来看看效果
rootNode.disp()

   pyramid   9
     eye   13
   pyramid   9
     eye   13
     phoenix   3


## 12.2.FP树的建立

In [3]:
def createTree(dataSet, minSup=1):
    """
    从数据集中建立FP树
    参数：
        dataSet -- 数据集
        minSup -- 最小支持度，整数，默认为1
    返回：
        retTree -- 构建好的FP树
        headerTable -- 头指针
    """
    #初始化头指针为空字典
    headerTable = {}
    #两次遍历数据集
    #第一次计算绝对频率
    #对于每个项目
    for trans in dataSet:
        #对于每个物体
        for item in trans:
            #统计出现次数，即支持度
            headerTable[item] = headerTable.get(item, 0) + dataSet[trans]
    #除去不满足最小支持度要求的关键词
    for k in list(headerTable.keys()):
        if headerTable[k] < minSup: 
            del(headerTable[k])
    #频繁集
    freqItemSet = set(headerTable.keys())
    #print('freqItemSet: ',freqItemSet)
    #如果频繁集为空集，那么直接退出
    if len(freqItemSet) == 0: return None, None
    #建立链表
    for k in headerTable:
        headerTable[k] = [headerTable[k], None]
    #print('headerTable: ',headerTable)
    #建立树
    retTree = treeNode('Null Set', 1, None)
    #第二次遍历数据集
    for tranSet, count in dataSet.items():
        #用于排序的频繁集
        localD = {}
        #填充频繁集
        for item in tranSet:
            if item in freqItemSet:
                localD[item] = headerTable[item][0]
        #如果频繁集不为空
        if len(localD) > 0:
            #排序，降序
            orderedItems = [v[0] for v in sorted(localD.items(), key=lambda p: p[1], reverse=True)]
            #更新树
            updateTree(orderedItems, retTree, headerTable, count)
    return retTree, headerTable

In [4]:
def updateTree(items, inTree, headerTable, count):
    """
    更新树
    参数：
        items -- 词条
        inTree -- 输入的树
        headerTable -- 链表
        count -- 数目，整数
    返回：
        无
    """
    #如果词条在输入树的子节点中
    if items[0] in inTree.children:
        #增加计数
        inTree.children[items[0]].inc(count)
    #如果没有，则增添子节点
    else:
        #新建子节点
        inTree.children[items[0]] = treeNode(items[0], count, inTree)
        #如果链表只有头节点
        if headerTable[items[0]][1] == None:
            #加入链表
            headerTable[items[0]][1] = inTree.children[items[0]]
        #更新链表
        else:
            updateHeader(headerTable[items[0]][1], inTree.children[items[0]])
    #递归调用函数，按照词条频率顺序更新子树
    if len(items) > 1:
        updateTree(items[1::], inTree.children[items[0]], headerTable, count)

In [5]:
def updateHeader(nodeToTest, targetNode):
    """
    更新链表
    参数：
        nodeToTest -- 待验证的节点
        targetNode -- 目标节点
    返回：
        无
    """
    #找到链表的末端
    while (nodeToTest.nodeLink != None):
        nodeToTest = nodeToTest.nodeLink
    #将节点加入链表
    nodeToTest.nodeLink = targetNode

## 12.3.简单数据集及数据包装器

In [6]:
def loadSimpDat():
    """
    读取简单数据集
    参数：
        无
    返回：
        simpDat -- 简单数据集，列表
    """
    simpDat = [['r', 'z', 'h', 'j', 'p'],
               ['z', 'y', 'x', 'w', 'v', 'u', 't', 's'],
               ['z'],
               ['r', 'x', 'n', 'o', 's'],
               ['y', 'r', 'x', 'z', 'q', 't', 'p'],
               ['y', 'z', 'x', 'e', 'q', 's', 't', 'm']]
    return simpDat

In [7]:
def createInitSet(dataSet):
    """
    初始化集
    参数：
        dataSet -- 数据集
    返回：
        retDict -- 返回包装后的数据集
    """
    #初始化包装的数据集为空字典
    retDict = {}
    #遍历数据集中的每一项事务，进行包装
    for trans in dataSet:
        retDict[frozenset(trans)] = 1
    return retDict

测试

In [8]:
simpDat = loadSimpDat()
initSet = createInitSet(simpDat)
print(f"格式化处理后的数据集：\n{initSet}")
myFPtree, myHeaderTab = createTree(initSet, 3)
myFPtree.disp()

格式化处理后的数据集：
{frozenset({'z', 'j', 'r', 'p', 'h'}): 1, frozenset({'y', 'z', 't', 's', 'w', 'u', 'x', 'v'}): 1, frozenset({'z'}): 1, frozenset({'s', 'n', 'o', 'r', 'x'}): 1, frozenset({'y', 'z', 't', 'q', 'r', 'p', 'x'}): 1, frozenset({'y', 'z', 's', 't', 'q', 'e', 'm', 'x'}): 1}
   Null Set   1
     z   5
       r   1
       x   3
         y   3
           t   2
             s   1
             r   1
           s   1
             t   1
     x   1
       s   1
         r   1


## 12.4.从一棵 FP树中挖掘频繁项集 

### 12.4.1.抽取条件模式基

In [9]:
def ascendTree(leafNode, prefixPath):
    """
    由叶节点到根节点的回溯函数
    参数：
        leafNode -- 叶节点
        prefixPath -- 前缀路径
    """
    #如果没有找到根节点
    if leafNode.parent != None:
        #把该节点添加到前缀路径中
        prefixPath.append(leafNode.name)
        #递归调用该回溯函数
        ascendTree(leafNode.parent, prefixPath)

In [10]:
def findPrefixPath(basePat, treeNode): 
    """
    寻找前缀路径
    参数：
        basePat -- 基路径
        treeNode -- 树节点
    返回：
        condPats -- 模式基字典
    """
    #初始化模式基字典为空字典
    condPats = {}
    #当树节点不为空时
    while treeNode != None:
        #初始化前缀路径为空列表
        prefixPath = []
        #找到前缀路径
        ascendTree(treeNode, prefixPath)
        #当前缀路径长度大于1时
        if len(prefixPath) > 1: 
            #记录前缀路径和对应的词条次数
            condPats[frozenset(prefixPath[1:])] = treeNode.count
        #沿着链表向后
        treeNode = treeNode.nodeLink
    return condPats

测试

In [11]:
print(f"x的模式基字典为：\n{findPrefixPath('x', myHeaderTab['x'][1])}")
print(f"z的模式基字典为：\n{findPrefixPath('z', myHeaderTab['z'][1])}")
print(f"r的模式基字典为：\n{findPrefixPath('r', myHeaderTab['r'][1])}")

x的模式基字典为：
{frozenset({'z'}): 3}
z的模式基字典为：
{}
r的模式基字典为：
{frozenset({'z'}): 1, frozenset({'s', 'x'}): 1, frozenset({'y', 't', 'z', 'x'}): 1}


### 12.4.2.创建条件FP树

In [19]:
def mineTree(inTree, headerTable, minSup, preFix, freqItemList):
    """
    递归查找频繁项集的递归函数
    参数：
        inTree -- 输入的FP树
        headerTable -- 头节点列表
        minSup -- 最小支持度
        preFix -- 前缀路径
        freqItemList -- 频繁项目列表
    返回：
        无
    """
    #排序头节点列表
    bigL = [v[0] for v in sorted(headerTable.items(), key=lambda p: p[1][0])]
    #从底部开始
    for basePat in bigL:
        newFreqSet = preFix.copy()
        newFreqSet.add(basePat)
        #print('finalFrequent Item: ',newFreqSet)
        #添加到频繁项列表中
        freqItemList.append(newFreqSet)
        #模式基字典
        condPattBases = findPrefixPath(basePat, headerTable[basePat][1])
        #print('condPattBases :',basePat, condPattBases)
        #建立条件FP树
        myCondTree, myHead = createTree(condPattBases, minSup)
        #print('head from conditional tree: ', myHead)
        #如果头节点不为空，挖掘条件FP树
        if myHead != None: #3. mine cond. FP-tree
            #print('conditional tree for: ',newFreqSet)
            #myCondTree.disp(1)
            #递归挖掘
            mineTree(myCondTree, myHead, minSup, newFreqSet, freqItemList)

测试

In [13]:
freqItems = []
mineTree(myFPtree, myHeaderTab, 3, set([]), freqItems)

conditional tree for:  {'y'}
   Null Set   1
     z   3
       x   3
conditional tree for:  {'y', 'x'}
   Null Set   1
     z   3
conditional tree for:  {'t'}
   Null Set   1
     y   3
       z   3
         x   3
conditional tree for:  {'t', 'z'}
   Null Set   1
     y   3
conditional tree for:  {'t', 'x'}
   Null Set   1
     y   3
       z   3
conditional tree for:  {'t', 'z', 'x'}
   Null Set   1
     y   3
conditional tree for:  {'s'}
   Null Set   1
     x   3
conditional tree for:  {'x'}
   Null Set   1
     z   3


## 12.5.示例：从新闻网站点击流中挖掘 

In [14]:
parsedDat = [line.split() for line in open('kosarak.dat').readlines()]
initSet = createInitSet(parsedDat)
myFPtree, myHeaderTab = createTree(initSet, 100000)

In [20]:
myFreqList = []
mineTree(myFPtree, myHeaderTab, 100000, set([]), myFreqList)
print(f"{len(myFreqList)}")
print(f"\n{myFreqList}")

9

[{'1'}, {'1', '6'}, {'3'}, {'11', '3'}, {'6', '11', '3'}, {'6', '3'}, {'11'}, {'11', '6'}, {'6'}]
